In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2regional as bwr
from pathlib import Path

In [ ]:
PROJECT_NAME = "Spain case study"

In [ ]:
bd.projects.set_current(PROJECT_NAME)

In [ ]:
meal = bd.get_node(name="meal")

In [ ]:
h2o = bd.get_node(name="water")

In [ ]:
data_dir = Path.cwd().absolute() / "data"
assert data_dir.is_dir()

In [ ]:
import fiona
from shapely.geometry import shape
from collections import defaultdict
import numpy as np

In [ ]:
ws = bwr.geocollections['water-stress']
regions = bwr.geocollections['regions']
cities = bwr.geocollections['cities']

cfs = defaultdict(list)

with fiona.open(ws['filepath']) as ws:
    for ws_feature in ws:
        ws_geometry = shape(ws_feature['geometry'])
        with fiona.open(regions['filepath']) as regs:
            for r_feature in regs:
                if ws_geometry.intersects(shape(r_feature['geometry'])):
                    cfs[('regions', r_feature.properties[regions['field']])].append(ws_feature.properties["ws2038cr"])
        with fiona.open(cities['filepath']) as cits:
            for c_feature in cits:
                if ws_geometry.intersects(shape(c_feature['geometry'])):
                    cfs[('cities', c_feature.properties[cities['field']])].append(ws_feature.properties["ws2038cr"])        

In [ ]:
formatted_cfs = [(h2o.key, np.average(values), where) for where, values in cfs.items()]

In [ ]:
bd.Method(("water stress", "shared_scale")).write(formatted_cfs)

In [ ]:
lca = bwr.OneSpatialScaleLCA({meal: 1}, ("water stress", "shared_scale"))
lca.lci()
lca.lcia()

In [ ]:
lca.score